In [17]:
# sql_related imports
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass
from Models.groq import llm
# langchain related imports
from langchain.tools import tool
from langchain.agents import create_agent
import os
from dotenv import load_dotenv
load_dotenv()
db_url=os.getenv("POSTGRES_URL")
db=SQLDatabase.from_uri(db_url)

# Adding memory

from langgraph.checkpoint.memory import MemorySaver



In [ ]:


@dataclass
class RuntimeContext:
    db:SQLDatabase

schema_info = db.get_table_info()


In [27]:
SYSTEM_PROMPT = f"""
You are a careful db analyst.

Database schema (authoritative — do NOT guess anything not listed):

Rules:
- This is a postgres db 
- If you dont know the tables then see it before making any decisions
- If you don't know the columns then use a command to get its column name before making any query or after making a error
- If you dont know the connections then see that too 
- Do NOT invent table or column names.
- When data is needed, call the tool `execute_sql` with ONE SELECT query.
- Use JOINs where required.
- Read-only queries only.
- Limit to 5 rows unless explicitly asked.
- Prefer explicit column lists.
- If a query fails, fix it USING THE SCHEMA ABOVE.
"""

In [20]:
def execute_query(db):
    @tool("execute_query")
    def perform_query(query:str):
        """
        This is will peform db operations 
        
        :param query: Description
        :type query: str
        """
        try:
            return db.run(query)
        except Exception as e:
            return f"Error occured ${e}"
    
    return perform_query
execute=execute_query(db)


In [28]:
agent=create_agent(llm,tools=[execute],system_prompt=SYSTEM_PROMPT,context_schema=RuntimeContext)


question = "list only tables names in the db and once done stop it"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

list only tables names in the db and once done stop it
================================== Ai Message ==================================
Tool Calls:
  execute_query (wc8rxg77z)
 Call ID: wc8rxg77z
  Args:
    query: SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';
================================= Tool Message =================================
Name: execute_query

[('roles_utility',), ('status_utility',), ('permission_role_map',), ('room_types',), ('rooms',), ('users',), ('permissions',), ('verifications',), ('sessions',), ('images',), ('room_amenities',), ('refund_room_map',), ('blacklisted_tokens',), ('reviews',), ('bookings',), ('tax_utility',), ('booking_room_map',), ('booking_tax_map',), ('edit_bookings',), ('issues',), ('payment_method_utility',), ('issue_chat',), ('wishlist',), ('notifications',), ('vw_customer_booking_summary',), ('vw_customer_payment_summary',), ('vw

In [21]:
memory_agent=create_agent(llm,tools=[execute],checkpointer=MemorySaver(),system_prompt=SYSTEM_PROMPT,context_schema=RuntimeContext)


In [29]:
for token in memory_agent.stream(
    {"messages":[{"role":"user","content":"count how many users are with having aswin in their name"}]},
    {"configurable":{"thread_id":"1"}},
    context=RuntimeContext(db),
        stream_mode="values",
    ):
    
    print(token["messages"][-1].pretty_print())


================================ Human Message =================================

count how many users are with having aswin in their name
None
================================== Ai Message ==================================
Tool Calls:
  execute_query (h07edshjm)
 Call ID: h07edshjm
  Args:
    query: SELECT column_name FROM information_schema.columns WHERE table_name = 'users';
  execute_query (akecmk3f2)
 Call ID: akecmk3f2
  Args:
    query: SELECT column_name FROM information_schema.columns WHERE table_name = 'users';
  execute_query (tj0fdq42s)
 Call ID: tj0fdq42s
  Args:
    query: SELECT COUNT(*) FROM users WHERE username LIKE '%Aswin%'
None
================================= Tool Message =================================
Name: execute_query

Error occured $(psycopg2.errors.UndefinedColumn) column "username" does not exist
LINE 1: SELECT COUNT(*) FROM users WHERE username LIKE '%Aswin%'
                                         ^

[SQL: SELECT COUNT(*) FROM users WHERE username L

In [30]:
for token in memory_agent.stream(
    {"messages":[{"role":"user","content":"get all of their phone numbers"}]},
    {"configurable":{"thread_id":"1"}},
    context=RuntimeContext(db),
        stream_mode="values",
    ):
    
    print(token["messages"][-1].pretty_print())


================================ Human Message =================================

get all of their phone numbers
None
================================== Ai Message ==================================
Tool Calls:
  execute_query (kqvssgmha)
 Call ID: kqvssgmha
  Args:
    query: SELECT phone_number FROM users WHERE full_name LIKE '%Aswin%'
None
================================= Tool Message =================================
Name: execute_query

[('8610476491',), ('6767676767',), ('8610471211',), ('9191919190',), ('9191919090',), ('8610471216',), ('8610471212',)]
None
================================== Ai Message ==================================

The phone numbers of the users with 'Aswin' in their name are as follows:

* 8610476491
* 6767676767
* 8610471211
* 9191919190
* 9191919090
* 8610471216
* 8610471212
None
